# Analysis of common aneuploidies

In [1]:
import pandas as pd

In [40]:
repeat_fraction_threshold = 1

In [74]:
cnv = pd.read_csv("/FastData/czirion/Crypto_Diversity_Pipeline/Crypto_Ashton/results_joined_2024-10-16/02.Dataset/cnv/cnv_calls.tsv", header=0, sep="\t")
chromosome_lengths = pd.read_csv("../data/chromosome_lengths.tsv", names=["Accession", "length"], sep="\t")
chromosome_names = pd.read_csv("/FastData/czirion/Crypto_Diversity_Pipeline/Crypto_Desjardins/config/chromosomes.csv", header=0, sep=",")

In [75]:
# Column names of chromosome names to uppercas
chromosome_names.columns = map(str.title, chromosome_names.columns)

In [76]:
cnv_filtered = cnv[(cnv["Repeat_fraction"] < repeat_fraction_threshold)]

In [77]:
cnv_chromosome = cnv_filtered.groupby(["Accession", "Sample", "CNV"]).agg({"Region_Size": "sum"}).reset_index().rename(columns={"Region_Size": "Total_CNV_size"})

In [78]:
cnv_percentage = cnv_chromosome.merge(chromosome_lengths, on="Accession")
cnv_percentage = cnv_percentage.merge(chromosome_names, on="Accession")
cnv_percentage["Percentage_CNV"] = ((cnv_percentage["Total_CNV_size"] / cnv_percentage["length"])* 100).round(2)

In [79]:
cnv_percentage

,Accession,Sample,CNV,Total_CNV_size,length,Lineage,Chromosome,Percentage_CNV
0,CP003820.1,ERS1142690,DELETION,77499,2291499,VNI,1,3.38
1,CP003820.1,ERS1142690,DUPLICATION,11000,2291499,VNI,1,0.48
2,CP003820.1,ERS1142691,DELETION,65999,2291499,VNI,1,2.88
3,CP003820.1,ERS1142691,DUPLICATION,21000,2291499,VNI,1,0.92
4,CP003820.1,ERS1142692,DELETION,41500,2291499,VNI,1,1.81
...,...,...,...,...,...,...,...,...
19438,chr14_Bt22,SRS885892,DELETION,12679,938179,VNBI,14,1.35
19439,chr14_Bt22,SRS885892,DUPLICATION,148500,938179,VNBI,14,15.83
19440,chr14_Bt22,SRS885893,DELETION,34179,938179,VNBI,14,3.64
19441,chr14_Bt22,SRS885894,DELETION,20679,938179,VNBI,14,2.20


In [88]:
whole_chromosome_cnv = cnv_percentage[cnv_percentage["Percentage_CNV"] >= 60]

In [94]:
summary = whole_chromosome_cnv.groupby(["Lineage", "Chromosome", "Accession", "CNV"]).agg({"Sample": "count"}).reset_index().rename(columns={"Sample": "Number_of_samples"})

In [95]:
summary.sort_values(by=["Number_of_samples"], ascending=False).reset_index(drop=True)

,Lineage,Chromosome,Accession,CNV,Number_of_samples
0,VNI,12,CP003831.1,DUPLICATION,16
1,VNI,13,CP003832.1,DUPLICATION,10
2,VNI,9,CP003828.1,DUPLICATION,5
3,VNI,6,CP003825.1,DUPLICATION,4
4,VNI,2,CP003821.1,DUPLICATION,3
5,VNI,4,CP003823.1,DUPLICATION,3
6,VNII,14,CP091260.1,DUPLICATION,3
7,VNBII,12,CP097935.1,DUPLICATION,2
8,VNI,1,CP003820.1,DUPLICATION,2
9,VNBI,9,chr09_Bt22,DUPLICATION,1
